In [ ]:
# DataFrames
import numpy as np
import pandas as pd
import polars as pl
from IPython.display import display, HTML

# Graphing utilities
import matplotlib.pyplot as plt
from matplotlib.dates import YearLocator, DateFormatter
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import graphs

# Database Connect
import db_connect
connection = db_connect.start_conn()

# General Data Exploration and Visualization

## Sale Counts

In [ ]:
# Getting Data
query_annualSales_byBoro = """
SELECT EXTRACT(YEAR FROM "SALE DATE") as year, 
       "BOROUGH",
       COUNT(*) as count
FROM rolling_sales
GROUP BY EXTRACT(YEAR FROM "SALE DATE"), "BOROUGH"
ORDER BY year, "BOROUGH"
"""
annual_byBoro = pl.read_database(query = query_annualSales_byBoro, connection = connection)

annual_byBoro_roc1 = annual_byBoro.sort(["BOROUGH", "year"]).with_columns(
    (pl.col("count").pct_change().over("BOROUGH")*100).alias("roc")
)
annual_sales=annual_byBoro.group_by("year").agg(pl.col("count").sum().alias("count")).sort("year")
annual_sales_roc1=annual_sales.sort("year").with_columns(
    (pl.col("count").pct_change()*100).alias("roc")
)

annual_byBoro_pd = annual_byBoro.to_pandas()
annual_byBoro_roc1_pd = annual_byBoro_roc1.to_pandas()
annual_sales_pd = annual_sales.to_pandas()
annual_sales_roc1_pd = annual_sales_roc1.to_pandas()

borough_names = {
    1: "Manhattan", 
    2: "Bronx", 
    3: "Brooklyn", 
    4: "Queens", 
    5: "Staten Island"
}

annual_byBoro_pd['borough_name'] = annual_byBoro_pd['BOROUGH'].map(borough_names)
annual_byBoro_roc1_pd['borough_name'] = annual_byBoro_roc1_pd['BOROUGH'].map(borough_names)

In [ ]:
query_monthlySales_byBoro = """
SELECT EXTRACT(MONTH FROM "SALE DATE") as month,
    "BOROUGH",
    COUNT(*) as count
FROM rolling_sales
GROUP BY EXTRACT(MONTH FROM "SALE DATE"), "BOROUGH"
ORDER BY month, "BOROUGH"
"""

monthly_byBoro = pl.read_database(query = query_monthlySales_byBoro, connection = connection)

monthly_byBoro_roc1 = monthly_byBoro.sort("month").with_columns(
    (pl.col("count").pct_change().over("BOROUGH")*100).alias("roc")
).to_pandas()
monthly_sales=monthly_byBoro.group_by("month").agg(pl.col("count").sum().alias("count")).sort("month")
monthly_sales_roc1=monthly_sales.sort("month").with_columns(
    (pl.col("count").pct_change()*100).alias("roc")
).to_pandas()

monthly_byBoro = monthly_byBoro.to_pandas()
monthly_sales = monthly_sales.to_pandas()

monthly_byBoro['borough_name'] = monthly_byBoro['BOROUGH'].map(borough_names)
monthly_byBoro_roc1['borough_name'] = monthly_byBoro_roc1['BOROUGH'].map(borough_names)

In [ ]:
query_annualSales_neighborhoods = """
SELECT EXTRACT(YEAR FROM "SALE DATE")::INTEGER as year,
    "NEIGHBORHOOD", 
    "BOROUGH",
    COUNT(*) as count
FROM rolling_sales
GROUP BY EXTRACT(YEAR FROM "SALE DATE"), "NEIGHBORHOOD", "BOROUGH"
ORDER BY year
"""

nborhood_annual_byBoro = pl.read_database(query = query_annualSales_neighborhoods, connection = connection)

nborhood_annual_byBoro_roc1 = nborhood_annual_byBoro.sort('year').with_columns(
    (pl.col('count').pct_change().over("NEIGHBORHOOD")*100).alias("roc")
).to_pandas()

nborhood_annual_byBoro = nborhood_annual_byBoro.to_pandas()

In [ ]:
fig, axs = plt.subplots(2,2)

graphs.MatPlot.makePlot(
    chart='line',
    data=annual_byBoro_pd,
    x='year',
    y='count',
    marker='o',
    hue='BOROUGH',
    palette=["C0", "C1", "C2", "C3", "C4"],
    title='Annual Sales by Borough',
    xticks=sorted(annual_byBoro_pd['year'].unique()),
    xlabel='Year',
    xrot=45,
    ylabel='Number of Sales',
    leg_on=True,
    leg_pos='upper right',
    leg_outer=False,
    leg_labels=annual_byBoro_pd['borough_name'].unique(),
    ax = axs[0,0]
)

graphs.MatPlot.makePlot(
    chart='line',
    data=annual_byBoro_roc1_pd,
    x='year',
    y='roc',
    marker='o',
    hue='BOROUGH',
    palette=["C0", "C1", "C2", "C3", "C4"],
    title='Annual Sales by Borough (Rate of Change 1 Period)',
    xticks=sorted(annual_byBoro_pd['year'].unique()),
    xlabel='Year',
    xrot=45,
    ylabel='Rate of Change %',
    leg_on=True,
    leg_pos='upper left',
    leg_outer=False,
    leg_labels=annual_byBoro_pd['borough_name'].unique(),
    ax = axs[0,1]
)

graphs.MatPlot.makePlot(
    chart='bar',
    data=annual_sales_pd,
    x='year',
    y='count',
    title='Annual Sales 2003-2023',
    xticks=[str(year) for year in annual_sales['year']],
    xlabel='Year',
    xrot=45,
    ylabel='Sales Count',
    leg_on=False,
    ax=axs[1,0]
)

graphs.MatPlot.makePlot(
    chart='line',
    data=annual_sales_roc1_pd,
    x='year',
    y='roc',
    marker='o',
    title='Annual Sales (Rate of Change 1 Period)',
    xticks=annual_sales_roc1_pd['year'],
    xlabel='Year',
    xrot=45,
    ylabel='Rate of Change %',
    leg_on=False,
    ax=axs[1,1]
)

# axs[1,0].annotate(
#     f'{annual_sales_pd["count"].min()}', 
#     (annual_sales_pd["count"].idxmin(), annual_sales_pd["count"].min())
# )

fig.set_figheight(14)
fig.set_figwidth(20)

In [ ]:
html = """
<div style="display: flex; gap: 25px">
    <div>{df1}</div>
    <div>{df2}</div>
    <div>{df3}</div>
    <div>{df4}</div>
</div>
""".format(
    df1=annual_byBoro_pd.nlargest(10,'count').to_html(), 
    df2=annual_byBoro_roc1_pd.nlargest(10,'roc').to_html(),
    df3=annual_sales_pd.nlargest(10,'count').to_html(),
    df4=annual_sales_roc1_pd.nlargest(10,'roc').to_html()
    )

display(HTML(html))

In [ ]:
fig, axs = plt.subplots(2,1)

graphs.MatPlot.makePlot(
    chart='bar',
    data=monthly_byBoro,
    x='month',
    y='count',
    title='Monthly Sales by Borough',
    hue='BOROUGH',
    palette=["C0", "C1", "C2", "C3", "C4"],
    # xticks=range(1,13),
    xlabel='Month',
    ylabel='Sales Count',
    leg_on=True,
    leg_pos='upper left',
    leg_outer=True,
    leg_bbox=(1,1),
    leg_labels=monthly_byBoro['borough_name'].unique(),
    ax = axs[0]
)

graphs.MatPlot.makePlot(
    chart='line',
    data=monthly_byBoro_roc1,
    x='month',
    y='roc',
    marker='o',
    title='Monthly Sales by Borough (ROC)',
    hue='BOROUGH',
    palette=["C0", "C1", "C2", "C3", "C4"],
    xlabel='Month',
    ylabel='Sales Count',
    leg_on=True,
    leg_pos='upper left',
    leg_outer=True,
    leg_bbox=(1,1),
    leg_labels=monthly_byBoro_roc1['borough_name'].unique(),
    ax = axs[1]
)

# plt.tight_layout()
fig.set_figheight(10)
fig.set_figwidth(8)

In [ ]:
html = """
<div style="display: flex; gap: 25px">
    <div>{df1}</div>
    <div>{df2}</div>
</div>
""".format(
    df1=monthly_byBoro.nlargest(10, 'count').to_html(), 
    df2=monthly_byBoro_roc1.nlargest(10, 'roc').to_html(),
    )

display(HTML(html))

In [ ]:
fig, axs = plt.subplots(2,1)

graphs.MatPlot.makePlot(
    chart='bar',
    data=monthly_sales,
    x='month',
    y='count',
    title='Property Sales by Month',
    xlabel='Month',
    ylabel='Sales Count',
    leg_on=False,
    ax = axs[0]
)

graphs.MatPlot.makePlot(
    chart='line',
    data=monthly_sales_roc1,
    x='month',
    y='roc',
    marker='o',
    title='Property Sales by Month (Rate of Change 1 Period)',
    xlabel='Month',
    ylabel='Rate of Change %',
    leg_on=False,
    ax = axs[1]
)

plt.tight_layout()

In [ ]:
html = """
<div style="display: flex; gap: 25px">
    <div>{df1}</div>
    <div>{df2}</div>
    <div>{df3}</div>
</div>
""".format(
    df1=nborhood_annual_byBoro.nlargest(10, 'count').to_html(), 
    df2=nborhood_annual_byBoro_roc1.nlargest(10, 'roc').to_html(),
    df3=(nborhood_annual_byBoro
         .sort_values(['year', 'count'], ascending=[True, False])
         .groupby('year')
         .first()
         .reset_index()
         ).to_html(),
         )

display(HTML(html))

## Economic Data

In [ ]:
query_overnight = "SELECT * FROM overnight_rates;"

overnight_rates = pl.read_database(query=query_overnight, connection=connection, infer_schema_length=None).to_pandas()

overnight_monthly = overnight_rates.groupby([overnight_rates['Effective Date'].dt.to_period('M'), 'Rate Type']).agg({'Rate (%)': 'mean', 'Volume ($Billions)': 'mean'})